# 드론 라이트 쇼 검색 트렌드 심화 분석

## 목표
- 지역별 검색 트렌드 비교 분석
- 시계열 패턴 및 계절성 분석
- 특이점(이벤트) 탐지 및 분석
- 상관관계 및 유사성 분석

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
from scipy.signal import find_peaks
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 스타일 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("whitegrid")

## 1. 데이터 로드

In [ ]:
# 전처리된 데이터 로드
df = pd.read_csv('../data/processed_data.csv')
df['date'] = pd.to_datetime(df['date'])

print(f"데이터 크기: {df.shape}")
print(f"기간: {df['date'].min()} ~ {df['date'].max()}")
print(f"지역: {list(df['region'].unique())}")

## 2. 지역별 트렌드 비교 분석

In [ ]:
# 지역별 데이터를 피벗 테이블로 변환
pivot_df = df.pivot(index='date', columns='region', values='ratio')
pivot_df.fillna(0, inplace=True)

print("지역별 피벗 데이터:")
print(pivot_df.head())
print(f"\n데이터 크기: {pivot_df.shape}")

In [ ]:
# 지역간 상관관계 분석
correlation_matrix = pivot_df.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.3f', cbar_kws={'shrink': 0.8})
plt.title('지역간 검색 트렌드 상관관계', fontsize=16, pad=20)
plt.tight_layout()
plt.savefig('../results/figures/correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("지역간 상관계수:")
print(correlation_matrix)

In [ ]:
# 월별 평균 트렌드 비교
monthly_trends = df.groupby(['month', 'region'])['ratio'].agg(['mean', 'std', 'max']).reset_index()

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 월별 평균
monthly_mean = monthly_trends.pivot(index='month', columns='region', values='mean')
monthly_mean.plot(kind='line', ax=axes[0,0], marker='o', linewidth=2)
axes[0,0].set_title('월별 평균 검색 비율', fontsize=14)
axes[0,0].set_xlabel('월')
axes[0,0].set_ylabel('평균 검색 비율')
axes[0,0].legend(title='지역')
axes[0,0].grid(True, alpha=0.3)

# 월별 최대값
monthly_max = monthly_trends.pivot(index='month', columns='region', values='max')
monthly_max.plot(kind='line', ax=axes[0,1], marker='s', linewidth=2)
axes[0,1].set_title('월별 최대 검색 비율', fontsize=14)
axes[0,1].set_xlabel('월')
axes[0,1].set_ylabel('최대 검색 비율')
axes[0,1].legend(title='지역')
axes[0,1].grid(True, alpha=0.3)

# 연도별 트렌드
yearly_trends = df.groupby(['year', 'region'])['ratio'].mean().reset_index()
sns.barplot(data=yearly_trends, x='year', y='ratio', hue='region', ax=axes[1,0])
axes[1,0].set_title('연도별 평균 검색 비율', fontsize=14)
axes[1,0].set_ylabel('평균 검색 비율')

# 분기별 트렌드
quarterly_trends = df.groupby(['quarter', 'region'])['ratio'].mean().reset_index()
sns.barplot(data=quarterly_trends, x='quarter', y='ratio', hue='region', ax=axes[1,1])
axes[1,1].set_title('분기별 평균 검색 비율', fontsize=14)
axes[1,1].set_xlabel('분기')
axes[1,1].set_ylabel('평균 검색 비율')

plt.tight_layout()
plt.savefig('../results/figures/temporal_trends.png', dpi=300, bbox_inches='tight')
plt.show()

## 3. 특이점(Peak) 탐지 및 분석

In [ ]:
# 각 지역별 피크 탐지
peak_analysis = {}

for region in pivot_df.columns:
    data = pivot_df[region].values
    
    # 피크 탐지 (높이 기준: 평균 + 2*표준편차)
    threshold = np.mean(data) + 2 * np.std(data)
    peaks, properties = find_peaks(data, height=threshold, distance=7)  # 최소 7일 간격
    
    peak_dates = pivot_df.index[peaks]
    peak_values = data[peaks]
    
    peak_analysis[region] = {
        'peaks': peaks,
        'dates': peak_dates,
        'values': peak_values,
        'threshold': threshold
    }
    
    print(f"\n{region} 주요 피크:")
    for date, value in zip(peak_dates, peak_values):
        print(f"  {date.strftime('%Y-%m-%d')}: {value:.4f}")

In [ ]:
# 피크 시각화
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=list(pivot_df.columns),
    shared_xaxes=True,
    vertical_spacing=0.08
)

colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

for i, region in enumerate(pivot_df.columns):
    # 원본 데이터
    fig.add_trace(
        go.Scatter(
            x=pivot_df.index,
            y=pivot_df[region],
            mode='lines',
            name=f'{region} 트렌드',
            line=dict(color=colors[i]),
            hovertemplate='%{x}<br>검색비율: %{y:.4f}<extra></extra>'
        ),
        row=i+1, col=1
    )
    
    # 피크 포인트
    peak_info = peak_analysis[region]
    fig.add_trace(
        go.Scatter(
            x=peak_info['dates'],
            y=peak_info['values'],
            mode='markers',
            name=f'{region} 피크',
            marker=dict(color='red', size=8, symbol='star'),
            hovertemplate='피크: %{x}<br>검색비율: %{y:.4f}<extra></extra>'
        ),
        row=i+1, col=1
    )
    
    # 임계값 선
    fig.add_hline(
        y=peak_info['threshold'],
        line_dash="dash",
        line_color="gray",
        annotation_text=f"임계값: {peak_info['threshold']:.4f}",
        row=i+1, col=1
    )

fig.update_layout(
    height=1000,
    title_text="지역별 검색 트렌드 및 주요 피크 분석",
    showlegend=True
)

fig.update_xaxes(title_text="날짜", row=3, col=1)
fig.update_yaxes(title_text="검색 비율")

fig.show()

## 4. 계절성 및 주기성 분석

In [ ]:
# 요일별 패턴 분석
df['day_of_week_num'] = df['date'].dt.dayofweek
day_names = ['월', '화', '수', '목', '금', '토', '일']

weekday_analysis = df.groupby(['day_of_week_num', 'region'])['ratio'].agg(['mean', 'std']).reset_index()

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 요일별 평균
weekday_mean = weekday_analysis.pivot(index='day_of_week_num', columns='region', values='mean')
weekday_mean.index = day_names
weekday_mean.plot(kind='bar', ax=axes[0], width=0.8)
axes[0].set_title('요일별 평균 검색 비율', fontsize=14)
axes[0].set_xlabel('요일')
axes[0].set_ylabel('평균 검색 비율')
axes[0].legend(title='지역')
axes[0].tick_params(axis='x', rotation=0)

# 시간대별 히트맵 (월-요일)
heatmap_data = df.groupby(['month', 'day_of_week_num', 'region'])['ratio'].mean().reset_index()
for i, region in enumerate(df['region'].unique()):
    region_data = heatmap_data[heatmap_data['region'] == region]
    pivot_heatmap = region_data.pivot(index='month', columns='day_of_week_num', values='ratio')
    pivot_heatmap.columns = day_names
    
    if i == 0:  # 첫 번째 지역만 히트맵으로 표시
        sns.heatmap(pivot_heatmap, annot=True, fmt='.3f', cmap='YlOrRd', ax=axes[1])
        axes[1].set_title(f'{region} - 월별/요일별 검색 패턴', fontsize=14)
        axes[1].set_xlabel('요일')
        axes[1].set_ylabel('월')

plt.tight_layout()
plt.savefig('../results/figures/seasonal_patterns.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. 통계적 분석

In [ ]:
# 지역간 검색 비율 차이 검정 (ANOVA)
regions = df['region'].unique()
region_data = [df[df['region'] == region]['ratio'].values for region in regions]

f_stat, p_value = stats.f_oneway(*region_data)

print("지역간 검색 비율 차이 검정 (ANOVA):")
print(f"F-통계량: {f_stat:.4f}")
print(f"p-값: {p_value:.6f}")
print(f"유의수준 0.05에서 {'유의한 차이 있음' if p_value < 0.05 else '유의한 차이 없음'}")

# 사후 검정 (Tukey HSD)
from scipy.stats import tukey_hsd

tukey_result = tukey_hsd(*region_data)
print(f"\nTukey HSD 사후 검정:")
print(tukey_result)

In [ ]:
# 트렌드 변화율 분석
trend_analysis = {}

for region in regions:
    region_data = pivot_df[region]
    
    # 7일 이동평균
    ma_7 = region_data.rolling(window=7).mean()
    
    # 30일 이동평균
    ma_30 = region_data.rolling(window=30).mean()
    
    # 변화율 계산
    pct_change_7 = ma_7.pct_change(periods=7) * 100
    pct_change_30 = ma_30.pct_change(periods=30) * 100
    
    trend_analysis[region] = {
        'ma_7': ma_7,
        'ma_30': ma_30,
        'pct_change_7': pct_change_7,
        'pct_change_30': pct_change_30
    }

# 이동평균 시각화
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=['이동평균 트렌드', '변화율 (%)']
)

colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

for i, region in enumerate(regions):
    # 이동평균
    fig.add_trace(
        go.Scatter(
            x=pivot_df.index,
            y=trend_analysis[region]['ma_30'],
            mode='lines',
            name=f'{region} (30일 MA)',
            line=dict(color=colors[i], width=2)
        ),
        row=1, col=1
    )
    
    # 변화율
    fig.add_trace(
        go.Scatter(
            x=pivot_df.index,
            y=trend_analysis[region]['pct_change_30'],
            mode='lines',
            name=f'{region} 변화율',
            line=dict(color=colors[i], width=1),
            showlegend=False
        ),
        row=2, col=1
    )

fig.update_layout(
    height=800,
    title_text="검색 트렌드 이동평균 및 변화율 분석"
)

fig.update_yaxes(title_text="검색 비율", row=1, col=1)
fig.update_yaxes(title_text="변화율 (%)", row=2, col=1)
fig.update_xaxes(title_text="날짜", row=2, col=1)

fig.show()

## 6. 분석 결과 요약

In [ ]:
# 분석 결과 요약 테이블 생성
summary_stats = []

for region in regions:
    region_data = pivot_df[region]
    peaks = peak_analysis[region]
    
    summary_stats.append({
        '지역': region,
        '평균 검색비율': region_data.mean(),
        '최대 검색비율': region_data.max(),
        '표준편차': region_data.std(),
        '피크 개수': len(peaks['peaks']),
        '최고 피크 날짜': region_data.idxmax().strftime('%Y-%m-%d'),
        '최고 피크 값': region_data.max(),
        '변동계수': region_data.std() / region_data.mean() if region_data.mean() > 0 else 0
    })

summary_df = pd.DataFrame(summary_stats)
summary_df = summary_df.round(4)

print("지역별 검색 트렌드 분석 요약:")
print(summary_df.to_string(index=False))

# 결과 저장
summary_df.to_csv('../results/reports/trend_analysis_summary.csv', index=False)
correlation_matrix.to_csv('../results/reports/correlation_analysis.csv')

print("\n분석 결과가 저장되었습니다.")